In [1]:
!pip install catboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [17]:
df = pd.read_csv('F:/fun/flights.csv')
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_2 = pd.read_csv('C:/Users/JARVIS/Desktop/unisys/airports.csv')
df_2.drop(['AIRPORT','COUNTRY','CITY','STATE'],axis=1,inplace=True)
df_2.rename(columns = {'IATA_CODE':'ORIGIN_AIRPORT'}, inplace = True)

In [19]:
df_2.head()

,ORIGIN_AIRPORT,LATITUDE,LONGITUDE
0,ABE,40.65236,-75.44040
1,ABI,32.41132,-99.68190
2,ABQ,35.04022,-106.60919
3,ABR,45.44906,-98.42183
4,ABY,31.53552,-84.19447


In [20]:
df = pd.merge(df,df_2,on=['ORIGIN_AIRPORT'])
df.rename(columns = {'LATITUDE':'origin_lat','LONGITUDE':'origin_lon'}, inplace = True)
df_2.rename(columns = {'ORIGIN_AIRPORT':'DESTINATION_AIRPORT','LATITUDE':'dest_lat','LONGITUDE':'dest_lon'}, inplace = True)
df = pd.merge(df,df_2,on=['DESTINATION_AIRPORT'])

In [21]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,origin_lat,origin_lon,dest_lat,dest_lon
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,NaN,NaN,NaN,NaN,NaN,61.17432,-149.99619,47.44898,-122.30931
1,2015,1,1,4,AS,108,N309AS,ANC,SEA,45,...,NaN,NaN,NaN,NaN,NaN,NaN,61.17432,-149.99619,47.44898,-122.30931
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,45,...,NaN,NaN,NaN,NaN,NaN,NaN,61.17432,-149.99619,47.44898,-122.30931
3,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,A,NaN,NaN,NaN,NaN,NaN,61.17432,-149.99619,47.44898,-122.30931
4,2015,1,1,4,AS,134,N464AS,ANC,SEA,155,...,NaN,NaN,NaN,NaN,NaN,NaN,61.17432,-149.99619,47.44898,-122.30931


In [11]:
df['ELAPSED_TIME'].mean()

137.19396401924632

In [22]:
df.drop(['TAXI_IN','WHEELS_ON','TAIL_NUMBER','CANCELLATION_REASON'],axis=1,inplace=True)
df.drop(['YEAR','CANCELLED','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'], axis = 1,inplace=True)
df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].astype(str)
df = df[df.ORIGIN_AIRPORT.apply(lambda x:True if len(x) == 3 else False)]
df.drop(['DAY','DAY_OF_WEEK','AIR_TIME','ARRIVAL_TIME','DIVERTED'],axis=1,inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332914 entries, 0 to 5332913
Data columns (total 19 columns):
MONTH                  int64
AIRLINE                object
FLIGHT_NUMBER          int64
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
SCHEDULED_DEPARTURE    int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
DISTANCE               int64
SCHEDULED_ARRIVAL      int64
ARRIVAL_DELAY          float64
origin_lat             float64
origin_lon             float64
dest_lat               float64
dest_lon               float64
dtypes: float64(11), int64(5), object(3)
memory usage: 813.7+ MB


In [24]:
df.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,origin_lat,origin_lon,dest_lat,dest_lon
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,1448,430,-22.0,61.17432,-149.99619,47.44898,-122.30931
1,1,AS,108,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,194.0,1448,509,-14.0,61.17432,-149.99619,47.44898,-122.30931
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,25.0,56.0,210.0,200.0,1448,515,-24.0,61.17432,-149.99619,47.44898,-122.30931
3,1,AS,136,ANC,SEA,135,NaN,NaN,NaN,NaN,205.0,NaN,1448,600,NaN,61.17432,-149.99619,47.44898,-122.30931
4,1,AS,134,ANC,SEA,155,140.0,-15.0,17.0,157.0,218.0,198.0,1448,633,-35.0,61.17432,-149.99619,47.44898,-122.30931


In [25]:
df.dropna(how='any',axis=0,inplace=True)
df.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,origin_lat,origin_lon,dest_lat,dest_lon
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,1448,430,-22.0,61.17432,-149.99619,47.44898,-122.30931
1,1,AS,108,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,194.0,1448,509,-14.0,61.17432,-149.99619,47.44898,-122.30931
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,25.0,56.0,210.0,200.0,1448,515,-24.0,61.17432,-149.99619,47.44898,-122.30931
4,1,AS,134,ANC,SEA,155,140.0,-15.0,17.0,157.0,218.0,198.0,1448,633,-35.0,61.17432,-149.99619,47.44898,-122.30931
5,1,AS,114,ANC,SEA,220,209.0,-11.0,15.0,224.0,200.0,199.0,1448,640,-12.0,61.17432,-149.99619,47.44898,-122.30931


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 32 columns):
YEAR                   0 non-null int64
MONTH                  0 non-null int64
DAY                    0 non-null int64
DAY_OF_WEEK            0 non-null int64
AIRLINE                0 non-null object
FLIGHT_NUMBER          0 non-null int64
TAIL_NUMBER            0 non-null object
ORIGIN_AIRPORT         0 non-null object
DESTINATION_AIRPORT    0 non-null object
SCHEDULED_DEPARTURE    0 non-null int64
DEPARTURE_TIME         0 non-null float64
DEPARTURE_DELAY        0 non-null float64
SCHEDULED_TIME         0 non-null float64
ELAPSED_TIME           0 non-null float64
AIR_TIME               0 non-null float64
DISTANCE               0 non-null int64
WHEELS_ON              0 non-null float64
TAXI_IN                0 non-null float64
SCHEDULED_ARRIVAL      0 non-null int64
ARRIVAL_TIME           0 non-null float64
DIVERTED               0 non-null int64
CANCELLED              0 non-null int64
CANCELL

In [14]:
#df_3 = df.groupby(['AIRLINE','DESTINATION_AIRPORT','ORIGIN_AIRPORT','MONTH'])[['ELAPSED_TIME']].mean()

In [15]:
#df_3.head()

ELAPSED_TIME
AIRLINE DESTINATION_AIRPORT ORIGIN_AIRPORT MONTH              
AA      ABQ                 CLT            7        213.645161
                                           8        221.064516
                                           9        217.333333
                            DFW            1        107.040650
                                           2        111.717949

In [15]:
#df.drop(['ELAPSED_TIME'],axis=1,inplace=True)

In [17]:
#df = pd.merge(df,df_3,on=['AIRLINE','DESTINATION_AIRPORT','ORIGIN_AIRPORT','MONTH'])

In [16]:
#df.head()

,MONTH,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,origin_lat,origin_lon,dest_lat_x,dest_lon_x,CITY_x,origin_lat,origin_lon,CITY_y,dest_lat_y,dest_lon_y
0,1,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,...,61.17432,-149.99619,47.44898,-122.30931,Anchorage,61.17432,-149.99619,Seattle,47.44898,-122.30931
1,1,AS,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,...,61.17432,-149.99619,47.44898,-122.30931,Anchorage,61.17432,-149.99619,Seattle,47.44898,-122.30931
2,1,DL,ANC,SEA,45,31.0,-14.0,25.0,56.0,210.0,...,61.17432,-149.99619,47.44898,-122.30931,Anchorage,61.17432,-149.99619,Seattle,47.44898,-122.30931
4,1,AS,ANC,SEA,155,140.0,-15.0,17.0,157.0,218.0,...,61.17432,-149.99619,47.44898,-122.30931,Anchorage,61.17432,-149.99619,Seattle,47.44898,-122.30931
5,1,AS,ANC,SEA,220,209.0,-11.0,15.0,224.0,200.0,...,61.17432,-149.99619,47.44898,-122.30931,Anchorage,61.17432,-149.99619,Seattle,47.44898,-122.30931


In [26]:
df_new = df
df_new.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,origin_lat,origin_lon,dest_lat,dest_lon
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,1448,430,-22.0,61.17432,-149.99619,47.44898,-122.30931
1,1,AS,108,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,194.0,1448,509,-14.0,61.17432,-149.99619,47.44898,-122.30931
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,25.0,56.0,210.0,200.0,1448,515,-24.0,61.17432,-149.99619,47.44898,-122.30931
4,1,AS,134,ANC,SEA,155,140.0,-15.0,17.0,157.0,218.0,198.0,1448,633,-35.0,61.17432,-149.99619,47.44898,-122.30931
5,1,AS,114,ANC,SEA,220,209.0,-11.0,15.0,224.0,200.0,199.0,1448,640,-12.0,61.17432,-149.99619,47.44898,-122.30931


In [29]:
df_new.drop(['ARRIVAL_DELAY','TAXI_OUT','WHEELS_OFF'],axis=1,inplace=True)

In [32]:
df

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat,dest_lon
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,1448,430,61.17432,-149.99619,47.44898,-122.30931
1,1,AS,108,ANC,SEA,45,41.0,-4.0,204.0,194.0,1448,509,61.17432,-149.99619,47.44898,-122.30931
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,210.0,200.0,1448,515,61.17432,-149.99619,47.44898,-122.30931
4,1,AS,134,ANC,SEA,155,140.0,-15.0,218.0,198.0,1448,633,61.17432,-149.99619,47.44898,-122.30931
5,1,AS,114,ANC,SEA,220,209.0,-11.0,200.0,199.0,1448,640,61.17432,-149.99619,47.44898,-122.30931
6,1,AS,730,ANC,SEA,505,457.0,-8.0,205.0,199.0,1448,930,61.17432,-149.99619,47.44898,-122.30931
7,1,AS,112,ANC,SEA,700,653.0,-7.0,210.0,202.0,1448,1130,61.17432,-149.99619,47.44898,-122.30931
8,1,AS,80,ANC,SEA,1035,1027.0,-8.0,207.0,206.0,1448,1502,61.17432,-149.99619,47.44898,-122.30931
9,1,AS,120,ANC,SEA,1340,1330.0,-10.0,195.0,195.0,1448,1755,61.17432,-149.99619,47.44898,-122.30931
10,1,AS,92,ANC,SEA,1430,1424.0,-6.0,200.0,189.0,1448,1850,61.17432,-149.99619,47.44898,-122.30931


In [21]:
df.drop(['target_departure'],axis=1,inplace=True)

KeyError: "['target_departure'] not found in axis"

In [33]:
df['target_departure'] = df['DEPARTURE_DELAY'].apply(lambda x: 0 if x<0 else (1 if x<30 else 2))
df.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat,dest_lon,target_departure
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,1448,430,61.17432,-149.99619,47.44898,-122.30931,0
1,1,AS,108,ANC,SEA,45,41.0,-4.0,204.0,194.0,1448,509,61.17432,-149.99619,47.44898,-122.30931,0
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,210.0,200.0,1448,515,61.17432,-149.99619,47.44898,-122.30931,0
4,1,AS,134,ANC,SEA,155,140.0,-15.0,218.0,198.0,1448,633,61.17432,-149.99619,47.44898,-122.30931,0
5,1,AS,114,ANC,SEA,220,209.0,-11.0,200.0,199.0,1448,640,61.17432,-149.99619,47.44898,-122.30931,0


In [34]:
df_zero = df[df['target_departure'] == 0]
df_zero.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat,dest_lon,target_departure
0,1,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,1448,430,61.17432,-149.99619,47.44898,-122.30931,0
1,1,AS,108,ANC,SEA,45,41.0,-4.0,204.0,194.0,1448,509,61.17432,-149.99619,47.44898,-122.30931,0
2,1,DL,1560,ANC,SEA,45,31.0,-14.0,210.0,200.0,1448,515,61.17432,-149.99619,47.44898,-122.30931,0
4,1,AS,134,ANC,SEA,155,140.0,-15.0,218.0,198.0,1448,633,61.17432,-149.99619,47.44898,-122.30931,0
5,1,AS,114,ANC,SEA,220,209.0,-11.0,200.0,199.0,1448,640,61.17432,-149.99619,47.44898,-122.30931,0


In [35]:
df_one = df[df['target_departure'] == 1]
df_one.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat,dest_lon,target_departure
17,1,AS,108,ANC,SEA,45,45.0,0.0,204.0,189.0,1448,509,61.17432,-149.99619,47.44898,-122.30931,1
27,1,AS,120,ANC,SEA,1340,1347.0,7.0,195.0,183.0,1448,1755,61.17432,-149.99619,47.44898,-122.30931,1
28,1,AS,92,ANC,SEA,1430,1439.0,9.0,200.0,180.0,1448,1850,61.17432,-149.99619,47.44898,-122.30931,1
29,1,DL,2436,ANC,SEA,1540,1552.0,12.0,211.0,192.0,1448,2011,61.17432,-149.99619,47.44898,-122.30931,1
38,1,AS,730,ANC,SEA,505,511.0,6.0,205.0,187.0,1448,930,61.17432,-149.99619,47.44898,-122.30931,1


In [36]:
df_two = df[df['target_departure'] == 2]
df_two.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat,dest_lon,target_departure
34,1,AS,108,ANC,SEA,45,138.0,53.0,204.0,171.0,1448,509,61.17432,-149.99619,47.44898,-122.30931,2
36,1,AS,136,ANC,SEA,135,224.0,49.0,205.0,180.0,1448,600,61.17432,-149.99619,47.44898,-122.30931,2
37,1,AS,134,ANC,SEA,155,228.0,33.0,218.0,188.0,1448,633,61.17432,-149.99619,47.44898,-122.30931,2
49,1,AS,98,ANC,SEA,5,41.0,36.0,205.0,188.0,1448,430,61.17432,-149.99619,47.44898,-122.30931,2
59,1,AS,80,ANC,SEA,1035,1136.0,61.0,207.0,188.0,1448,1502,61.17432,-149.99619,47.44898,-122.30931,2


In [37]:
df_zero_sample = df_zero.sample(n=600000,axis=0)
df_one_sample = df_one.sample(n=600000,axis=0)
df_two_sample = df_two.sample(n=600000,axis=0)

In [38]:
df = pd.concat([df_zero_sample,df_one_sample,df_two_sample],ignore_index=True)

In [39]:
df.drop(['DEPARTURE_DELAY'],axis=1, inplace=True)

In [20]:
df.head()

,MONTH,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,STATE_x,origin_lat,origin_lon,STATE_y,dest_lat,dest_lon,target_departure
0,7,DL,SRQ,ATL,555,552.0,93.0,84.0,444,728,FL,27.39533,-82.55411,GA,33.64044,-84.42694,0
1,11,OO,ORD,MKG,910,906.0,55.0,76.0,118,1105,IL,41.97960,-87.90446,MI,43.16949,-86.23822,0
2,4,OO,SPI,ORD,605,555.0,59.0,54.0,174,704,IL,39.84395,-89.67762,IL,41.97960,-87.90446,0
3,11,DL,IND,MSP,1609,1605.0,106.0,101.0,503,1655,IN,39.71733,-86.29438,MN,44.88055,-93.21692,0
4,6,B6,LGB,AUS,700,657.0,168.0,156.0,1226,1148,CA,33.81772,-118.15161,TX,30.19453,-97.66987,0


In [40]:
X = df.drop(['target_departure'],axis=1)
y = df['target_departure']

In [41]:
X.drop(['DEPARTURE_TIME','origin_lat','origin_lon','dest_lat','dest_lon'],axis=1,inplace=True)
X.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,4,AS,50,BRW,ANC,1855,108.0,105.0,725,2043
1,7,WN,4841,SJU,FLL,700,160.0,151.0,1046,940
2,2,DL,2696,LGA,BOS,2159,67.0,76.0,184,2306
3,8,OO,6503,SFO,EUG,1950,98.0,89.0,451,2128
4,5,OO,5649,FAT,DEN,1322,142.0,131.0,844,1644


In [42]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state = 0)

In [43]:
len(df['ORIGIN_AIRPORT'].unique())

319

In [33]:
df[df['target_departure']==2]

,MONTH,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,origin_lat,origin_lon,dest_lat_x,dest_lon_x,CITY_x,origin_lat,origin_lon,CITY_y,dest_lat_y,dest_lon_y,target_departure
1200000,5,DL,ATL,CHS,2256,2351.0,69.0,259,5,33.64044,-84.42694,32.89865,-80.04051,Atlanta,33.64044,-84.42694,Charleston,32.89865,-80.04051,2
1200001,7,EV,ATL,MOB,1915,2120.0,77.0,302,1932,33.64044,-84.42694,30.69142,-88.24283,Atlanta,33.64044,-84.42694,Mobile,30.69142,-88.24283,2
1200002,3,EV,DFW,GRK,1745,1932.0,45.0,134,1830,32.89595,-97.03720,31.06490,-97.82780,Dallas-Fort Worth,32.89595,-97.03720,Killeen,31.06490,-97.82780,2
1200003,7,AA,DFW,PHX,820,854.0,157.0,868,857,32.89595,-97.03720,33.43417,-112.00806,Dallas-Fort Worth,32.89595,-97.03720,Phoenix,33.43417,-112.00806,2
1200004,12,DL,SEA,LAX,1459,1551.0,161.0,954,1740,47.44898,-122.30931,33.94254,-118.40807,Seattle,47.44898,-122.30931,Los Angeles,33.94254,-118.40807,2
1200005,12,AA,MCI,CLT,1410,1530.0,129.0,808,1719,39.29761,-94.71391,35.21401,-80.94313,Kansas City,39.29761,-94.71391,Charlotte,35.21401,-80.94313,2
1200006,11,AA,CLT,MIA,2230,2313.0,121.0,650,31,35.21401,-80.94313,25.79325,-80.29056,Charlotte,35.21401,-80.94313,Miami,25.79325,-80.29056,2
1200007,8,WN,BNA,PNS,2025,2057.0,75.0,391,2140,36.12448,-86.67818,30.47331,-87.18744,Nashville,36.12448,-86.67818,Pensacola,30.47331,-87.18744,2
1200008,1,AS,OAK,SEA,1305,1335.0,115.0,671,1500,37.72129,-122.22072,47.44898,-122.30931,Oakland,37.72129,-122.22072,Seattle,47.44898,-122.30931,2
1200009,7,MQ,ORD,COU,1418,1547.0,80.0,315,1538,41.97960,-87.90446,38.81809,-92.21963,Chicago,41.97960,-87.90446,Columbia,38.81809,-92.21963,2


In [44]:
X_train.head()

,MONTH,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL
774454,7,AA,295,TPA,MIA,710,65.0,58.0,204,815
670962,9,MQ,3261,ORD,OMA,1330,97.0,106.0,416,1507
801955,2,OO,2600,LAX,RDM,1750,125.0,132.0,726,1955
1395977,12,DL,2161,ATL,PHL,1635,126.0,121.0,666,1841
960024,4,F9,1300,ORD,MIA,630,180.0,166.0,1197,1030


In [45]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
cat_features=[0,1,2,3,4]
model = CatBoostClassifier(iterations=10,
                           depth=2,
                           learning_rate=1,
                           loss_function='MultiClass',
                           verbose=True)
# train the model
model.fit(X_train,y_train, cat_features)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class)
print("proba = ", preds_proba)
print("accuracy = ", accuracy_score(y_test,preds_class))

0:	learn: 1.0672317	total: 2.77s	remaining: 24.9s
1:	learn: 1.0603128	total: 3.87s	remaining: 15.5s
2:	learn: 1.0543715	total: 4.92s	remaining: 11.5s
3:	learn: 1.0514840	total: 5.7s	remaining: 8.56s
4:	learn: 1.0496027	total: 6.61s	remaining: 6.61s
5:	learn: 1.0483076	total: 7.51s	remaining: 5.01s
6:	learn: 1.0464542	total: 8.38s	remaining: 3.59s
7:	learn: 1.0453926	total: 9.19s	remaining: 2.3s
8:	learn: 1.0444769	total: 9.94s	remaining: 1.1s
9:	learn: 1.0437621	total: 10.7s	remaining: 0us
class =  [[1.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [2.]]
proba =  [[0.18461357 0.43486847 0.38051795]
 [0.65270615 0.19891722 0.14837663]
 [0.3448264  0.44727448 0.20789913]
 ...
 [0.44752597 0.35645342 0.19602061]
 [0.68151071 0.19515528 0.12333401]
 [0.22341996 0.36373302 0.41284702]]
accuracy =  0.4527


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() 
df['AIRLINE']= le.fit_transform(df['AIRLINE'])
df['ORIGIN_AIRPORT']= le.fit_transform(df['ORIGIN_AIRPORT']) 
df['DESTINATION_AIRPORT'] = le.fit_transform(df['DESTINATION_AIRPORT'])

In [36]:
import xgboost as xgb
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations
model = xgb.train(param, D_train, steps)
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

ValueError: ignored